In [ ]:
# FOR BACKUP ONLY

In [1]:
import logging
import pickle

import string

import pandas as pd 
import numpy as np 
import re
import os.path

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords

In [2]:
spam_data = pd.read_csv("../data/spam_cleaned.csv", encoding='latin-1')

In [3]:
spam_data[spam_data['label'] == 'ham'].sample(5)

,Unnamed: 0,label,text,spam,length,filtered_text
64,64,ham,Ok lar i double check wif da hair dresser alre...,0,110,ok lar double check wif da hair dresser alread...
2239,2239,ham,Every day i use to sleep after &lt;#&gt; so ...,0,51,every day use sleep ltgt
5149,5149,ham,Pls call me da. What happen.,0,28,pls call da happen
1132,1132,ham,Good morning princess! How are you?,0,35,good morning princess
4557,4557,ham,PISS IS TALKING IS SOMEONE THAT REALISE U THAT...,0,90,piss talking someone realise u point itnow rea...


In [4]:
# Train Test Split
# logger.info('Preparing Train/Test Set...')
corpus = spam_data.filtered_text.values.astype('U')
y = spam_data.spam.values
Xc_train, Xc_test, y_train, y_test = train_test_split(corpus, y, test_size=0.25)

In [5]:
Xc_train

array(['wat r u',
       'madamregret disturbancemight receive reference check dlf premaricakindly informedrgdsrakheshkerala',
       'good afternoon starshine hows boytoy crave yet ache fuck sips cappuccino miss babe teasing kiss',
       ...,
       'erm woodland avenue somewhere get parish magazine telephone number',
       'sfirst timedhoni rocks',
       'sounds good fingers makes difficult type'], dtype='<U516')

In [6]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
# logger.info('Vectorizing Training Set')
tf = TfidfVectorizer(use_idf=True)
Xctr_tf_fitted = tf.fit(Xc_train)
# should save Xct_tf_fitted in pickle for vocab list
Xctr_tf_fitted_txform = Xctr_tf_fitted.transform(Xc_train)

In [10]:
Xc_train[0]

'wat r u'

In [11]:
first_vec=Xctr_tf_fitted_txform[0]
df = pd.DataFrame(first_vec.T.todense(), index=tf.get_feature_names(), columns=['tfidf'])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
wat,1.0
008704050406,0.0
perhaps,0.0
personality,0.0
personal,0.0
...,...
ends,0.0
endowed,0.0
endless,0.0
ending,0.0


In [12]:
# logger.info('Fitting Training Set into Linear SVC')
algo = LinearSVC()
algo.fit(Xctr_tf_fitted_txform, y_train)

LinearSVC()

In [13]:
# Vectorizing Test Set
tf_test = TfidfVectorizer(use_idf=True, vocabulary=tf.vocabulary_)
Xcte_tf_fitted_txform = tf_test.fit_transform(Xc_test)

In [14]:
Xc_test[0]

'cant pick phone right pls send message'

In [15]:
first_vec=Xcte_tf_fitted_txform[0]
df = pd.DataFrame(first_vec.T.todense(), index=tf_test.get_feature_names(), columns=['tfidf'])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
pick,0.404234
pls,0.393043
right,0.389650
message,0.377410
cant,0.374634
...,...
ends,0.000000
endowed,0.000000
endless,0.000000
ending,0.000000


In [18]:
yt_pred_LSVC = algo.predict(Xcte_tf_fitted_txform)
LSVC_acc = accuracy_score(y_test, yt_pred_LSVC)
msg = 'Finalizing Vectorizer, Linear SVC Accuracy: ' + str(LSVC_acc)
print(msg)
#logger.info(msg)

Finalizing Vectorizer, Linear SVC Accuracy: 0.9849246231155779


In [ ]:
# Evaluate Test Set